In [88]:
# import libraries
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [89]:
# function to load one batch
def getData(file) :
    name = './cifar-10-batches-py/'+file
    with open(name, 'rb') as fo:
        data = pickle.load(fo, encoding="bytes")
    return data

In [90]:
# loading train-validation data
X = []
y = []
for i in range(5) :
    filename = "data_batch_"+str(i+1)
    data = getData(filename)
    X.append(data[b'data'])
    y.append(data[b'labels'])
# input of size (50000, 3072) and output of size (50000,1)
X = np.array(X).reshape(-1,3072)
X = np.divide(X,255)
y = np.array(y).reshape(50000)

In [91]:
# loading test data
Xtest = getData("test_batch")[b'data']
ytest = np.array(getData("test_batch")[b'labels'])
Xtest = np.divide(Xtest,255)

In [92]:
label_names = [lbl.decode("utf-8") for lbl in getData("batches.meta")[b'label_names']]

In [93]:
# using airplane, automobile and bird classes
Xnew11 = []
ynew11 = []
Xnew12 = []
ynew12 = []
Xnew10 = []
ynew10 = []

for i in range(len(y)) :
    if y[i]==0 :
        Xnew10.append(X[i])
        ynew10.append(y[i])
    elif y[i]==1 :
        Xnew11.append(X[i])
        ynew11.append(y[i])
    elif y[i]==2 :
        Xnew12.append(X[i])
        ynew12.append(y[i])
    
Xnew10 = np.array(Xnew10)
ynew10 = np.array(ynew10)
Xnew11 = np.array(Xnew11)
ynew11 = np.array(ynew11)
Xnew12 = np.array(Xnew12)
ynew12 = np.array(ynew12)

In [94]:
Xnew2 = []
ynew2 = []
for i in range(len(ytest)) :
    if ytest[i] in [0,1,2] :
        Xnew2.append(Xtest[i])
        ynew2.append(ytest[i])
Xnew2 = np.array(Xnew2)
ynew2 = np.array(ynew2)


In [95]:
# function to shuffle arrays
def shuffleArrays(a, b):
    assert len(a) == len(b)
    index = np.random.permutation(len(a))
    return a[index], b[index]

In [96]:
[x_train0,x_test0,y_train0,y_test0] = train_test_split(Xnew10,ynew10,random_state=0,train_size=0.8)
[x_train1,x_test1,y_train1,y_test1] = train_test_split(Xnew11,ynew11,random_state=0,train_size=0.8)
[x_train2,x_test2,y_train2,y_test2] = train_test_split(Xnew12,ynew12,random_state=0,train_size=0.8)


/home/meenu/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [97]:
def doClassifier(X1,y1,X2,y2,X3,y3,X4,y4) :
    xtrain = []
    ytrain = []
    xvalid = []
    yvalid = []
    
    xtrain.append(X1)
    xtrain.append(X2)
    ytrain.append(y1)
    ytrain.append(y2)
    
    xvalid.append(X3)
    xvalid.append(X4)
    yvalid.append(y3)
    yvalid.append(y4)
    
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xvalid = np.array(xvalid)
    yvalid = np.array(yvalid)
    
    xtrain = xtrain.reshape(-1,3072)
    xvalid = xvalid.reshape(-1,3072)
    ytrain = ytrain.reshape(xtrain.shape[0])
    yvalid = yvalid.reshape(xvalid.shape[0])
    
    
    
    xtrain, ytrain = shuffleArrays(xtrain,ytrain)
    xvalid, yvalid = shuffleArrays(xvalid,yvalid)
    
    return xtrain,xvalid,ytrain,yvalid

In [98]:
train_time_ovo = 0
train_time_ova = 0


In [99]:
# getting data for the first classifier
xtrain01,xvalid01,ytrain01,yvalid01 = doClassifier(x_train0, y_train0,x_train1, y_train1,x_test0, y_test0,x_test1, y_test1)
# model for classifier - classes 0 and 1 
max_iters = 100

start = time.time()*1000
clf01 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain01,ytrain01)
end = time.time()*1000
train_time_ovo += (end - start)

print("Train score:{:.2f}".format(clf01.score(xtrain01,ytrain01)))
print("Validation score:{:.2f}".format(clf01.score(xvalid01,yvalid01)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.85
Validation score:0.80


In [100]:
# getting data for the first classifier
xtrain02,xvalid02,ytrain02,yvalid02 = doClassifier(x_train0, y_train0,x_train2, y_train2,x_test0, y_test0,x_test2, y_test2)
max_iters = 100

start = time.time()*1000
clf02 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain02,ytrain02)
end = time.time()*1000
train_time_ovo += (end-start)
print("Train score:{:.2f}".format(clf02.score(xtrain02,ytrain02)))
print("Validation score:{:.2f}".format(clf02.score(xvalid02,yvalid02)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.83
Validation score:0.77


In [101]:
# getting data for the first classifier
xtrain12,xvalid12,ytrain12,yvalid12 = doClassifier(x_train1, y_train1,x_train2, y_train2,x_test1, y_test1,x_test2, y_test2)
max_iters = 100

start = time.time()*1000
clf12 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain12,ytrain12)
end = time.time()*1000
train_time_ovo += (end-start)
print("Train score:{:.2f}".format(clf12.score(xtrain12,ytrain12)))
print("Validation score:{:.2f}".format(clf12.score(xvalid12,yvalid12)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.88
Validation score:0.83


In [102]:
# computing accuracy
pred01 = clf01.predict_proba(Xnew2)
pred02 = clf02.predict_proba(Xnew2)
pred12 = clf12.predict_proba(Xnew2)

ypred = np.zeros((Xnew2.shape[0],3))

for i in range(Xnew2.shape[0]) :
    ypred[i][0] += pred01[i][0]
    ypred[i][0] += pred02[i][0]

for i in range(Xnew2.shape[0]) :
    ypred[i][1] += pred01[i][1]
    ypred[i][1] += pred12[i][0]

    
for i in range(Xnew2.shape[0]) :
    ypred[i][2] += pred02[i][1]
    ypred[i][2] += pred12[i][1]

yfinal = np.argmax(ypred,axis=1)



In [103]:
# Accuracy of ovo classifier
print("Test score:{:.4f}".format(accuracy_score(yfinal,ynew2)))
# Computation time
print("Computation time of training OVO classifier in milliseconds ",train_time_ovo)

Test score:0.7047
Computation time of training OVO classifier in milliseconds  68829.89404296875


In [104]:
def doOVAClassifier0(X1,y1,X2,y2,X3,y3,X4,y4,X5,y5,X6,y6) :
    xtrain = []
    ytrain = []
    xvalid = []
    yvalid = []
    
    xtrain.append(X1)
    xtrain.append(X2)
    xtrain.append(X3)
    # 0 and 1 labels
    ytrain.append(y1)
    ytrain.append(y2)
    ytrain.append(y2)
    
    xvalid.append(X4)
    xvalid.append(X5)
    xvalid.append(X6)
    
    yvalid.append(y4)
    yvalid.append(y5)
    yvalid.append(y5)
    
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xvalid = np.array(xvalid)
    yvalid = np.array(yvalid)
    
    xtrain = xtrain.reshape(-1,3072)
    xvalid = xvalid.reshape(-1,3072)
    ytrain = ytrain.reshape(xtrain.shape[0])
    yvalid = yvalid.reshape(xvalid.shape[0])
    
    
    
    xtrain, ytrain = shuffleArrays(xtrain,ytrain)
    xvalid, yvalid = shuffleArrays(xvalid,yvalid)
    
    return xtrain,xvalid,ytrain,yvalid

In [105]:
xtrain0,xvalid0,ytrain0,yvalid0 = doOVAClassifier0(x_train0, y_train0, x_train1, y_train1, x_train2,y_train2, x_test0, y_test0, x_test1, y_test1, x_test2, y_test2)

max_iters = 100

start = time.time()*1000
clf0 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain0,ytrain0)
end = time.time()*1000
train_time_ova += (end-start)
print("Train score:{:.2f}".format(clf0.score(xtrain0,ytrain0)))
print("Validation score:{:.2f}".format(clf0.score(xvalid0,yvalid0)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.82
Validation score:0.76


In [106]:
def doOVAClassifier1(X1,y1,X2,y2,X3,y3,X4,y4,X5,y5,X6,y6) :
    xtrain = []
    ytrain = []
    xvalid = []
    yvalid = []
    
    xtrain.append(X1)
    xtrain.append(X2)
    xtrain.append(X3)
    # 1 and 0 labels
    ytrain.append(y1)
    ytrain.append(y2)
    ytrain.append(y1)
    
    xvalid.append(X4)
    xvalid.append(X5)
    xvalid.append(X6)
    
    yvalid.append(y4)
    yvalid.append(y5)
    yvalid.append(y4)
    
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xvalid = np.array(xvalid)
    yvalid = np.array(yvalid)
    
    xtrain = xtrain.reshape(-1,3072)
    xvalid = xvalid.reshape(-1,3072)
    ytrain = ytrain.reshape(xtrain.shape[0])
    yvalid = yvalid.reshape(xvalid.shape[0])
    
    
    
    xtrain, ytrain = shuffleArrays(xtrain,ytrain)
    xvalid, yvalid = shuffleArrays(xvalid,yvalid)
    
    return xtrain,xvalid,ytrain,yvalid

In [107]:
xtrain1,xvalid1,ytrain1,yvalid1 = doOVAClassifier1(x_train0, y_train0, x_train1, y_train1, x_train2,y_train2, x_test0, y_test0, x_test1, y_test1, x_test2, y_test2)

max_iters = 100

start = time.time()*1000
clf1 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain1,ytrain1)
end = time.time()*1000
train_time_ova += (end-start)
print("Train score:{:.2f}".format(clf1.score(xtrain1,ytrain1)))
print("Validation score:{:.2f}".format(clf1.score(xvalid1,yvalid1)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.86
Validation score:0.83


In [108]:
def doOVAClassifier2(X1,y1,X2,y2,X3,y3,X4,y4,X5,y5,X6,y6) :
    xtrain = []
    ytrain = []
    xvalid = []
    yvalid = []
    
    xtrain.append(X1)
    xtrain.append(X2)
    xtrain.append(X3)
    # 2 and 0 labels
    ytrain.append(y1)
    ytrain.append(y1)
    ytrain.append(y3)
    
    xvalid.append(X4)
    xvalid.append(X5)
    xvalid.append(X6)
    
    yvalid.append(y4)
    yvalid.append(y4)
    yvalid.append(y6)
    
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xvalid = np.array(xvalid)
    yvalid = np.array(yvalid)
    
    xtrain = xtrain.reshape(-1,3072)
    xvalid = xvalid.reshape(-1,3072)
    ytrain = ytrain.reshape(xtrain.shape[0])
    yvalid = yvalid.reshape(xvalid.shape[0])
    
    
    
    xtrain, ytrain = shuffleArrays(xtrain,ytrain)
    xvalid, yvalid = shuffleArrays(xvalid,yvalid)
    
    return xtrain,xvalid,ytrain,yvalid

In [109]:
xtrain2,xvalid2,ytrain2,yvalid2 = doOVAClassifier2(x_train0, y_train0, x_train1, y_train1, x_train2,y_train2, x_test0, y_test0, x_test1, y_test1, x_test2, y_test2)

max_iters = 100

start = time.time()*1000
clf2 = LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100).fit(xtrain2,ytrain2)
end = time.time()*1000
train_time_ova += (end-start)
print("Train score:{:.2f}".format(clf2.score(xtrain2,ytrain2)))
print("Validation score:{:.2f}".format(clf2.score(xvalid2,yvalid2)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Train score:0.84
Validation score:0.80


In [110]:
# computing accuracy
pred0 = clf0.predict_proba(Xnew2)
pred1 = clf1.predict_proba(Xnew2)
pred2 = clf2.predict_proba(Xnew2)



ypred = np.zeros((Xnew2.shape[0],3))

for i in range(Xnew2.shape[0]) :
    ypred[i][0] = pred0[i][0]

for i in range(Xnew2.shape[0]) :
    ypred[i][1] = pred1[i][1]

    
for i in range(Xnew2.shape[0]) :
    ypred[i][2] = pred2[i][1]

yfinal = np.argmax(ypred,axis=1)


In [111]:
# Accuracy of ova classifier
print("Test score:{:.4f}".format(accuracy_score(yfinal,ynew2)))
# Computation time
print("Computation time of training OVO classifier in milliseconds ",train_time_ova)

Test score:0.7030
Computation time of training OVO classifier in milliseconds  96720.5947265625


### Results for Q1 ###

1. The time required for training one vs all is more than one vs one in the case of 3 classes. This is because the number of samples per classifier are more in case of OVA although both have 3 classifiers to be trained. This is not the case when more than 3 classes are present, as the number of combinations of ovo classifiers are of the order n^2 whereas ova has only n, where n is the number of classes. Hence OVA is preferred when n is high.

2. Accuracy of both the classfiers on 10,000 samples of test data are almost the same.

3. Probabilities of predictions have been taken into account for max-voting, so that ties won't arise among class labels

4. max_iters were varied as 100,200,400. But accuracy on validation set decreased as max_iters was increased. Hence max_iters = 100 was used.

In [122]:
# one vs all classifier for 10 classes
from sklearn.multiclass import OneVsRestClassifier
[xTrain,xTest,yTrain,yTest] = train_test_split(X,y,random_state=0,train_size=0.8)

start = time.time() * 1000
clfOva = OneVsRestClassifier(LogisticRegression(solver='sag',n_jobs = -1, max_iter=max_iters, C=100)).fit(xTrain, yTrain)
end = time.time() * 1000
print("Computation time required in milliseconds",end-start)

yPred = clfOva.predict(xTrain)
print("Train score:{:.4f}".format(accuracy_score(yPred,yTrain)))
yPred = clfOva.predict(xTest)
print("Validation score:{:.4f}".format(accuracy_score(yPred,yTest)))
yPred = clfOva.predict(Xtest)
print("Validation score:{:.4f}".format(accuracy_score(yPred,ytest)))


/home/meenu/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/meenu/.local/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the co

Computation time required in milliseconds 1242165.04296875
Train score:0.4784
Validation score:0.3906
Validation score:0.3850
